# CSE 158/258, Fall 2020: Homework 2

## Instructions
Please submit your solution **by the beginning of the week 5 lecture (Nov 2)**. Submissions should be
made on **gradescope**. Please complete homework **individually**.

This specification includes both questions from the undergraduate (CSE158) and graduate (CSE258) classes.
You are welcome to attempt questions from both classes but will only be graded on those for the class in which
you are enrolled.

You will need the following files:

**Beer Reviews** : https://cseweb.ucsd.edu/classes/fa20/cse258-a/data/beer_50000.json

**Facebook ego network** : https://cseweb.ucsd.edu/classes/fa20/cse258-a/data/egonet.txt.

**Code examples** : http://cseweb.ucsd.edu/classes/fa19/cse258-a/code/week2.py (classification) and
http://cseweb.ucsd.edu/classes/fa19/cse258-a/code/week3.py (clustering/communities)

Executing the code requires a working install of Python 2.7 or Python 3 with the scipy packages installed.
**Please include the code of (the important parts of) your solutions**.

In [4]:
#Written in python 3
import numpy as np
from urllib.request import urlopen
import scipy.optimize
import random
import ast
import os
import requests
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model, svm
from sklearn import linear_model, metrics
import gzip
import re
from IPython.display import display, Markdown

In [5]:
beer_url = 'https://cseweb.ucsd.edu/classes/fa20/cse258-a/data/beer_50000.json'
fb_url = 'https://cseweb.ucsd.edu/classes/fa20/cse258-a/data/egonet.txt'
ex1_url = 'http://cseweb.ucsd.edu/classes/fa19/cse258-a/code/week2.py'
ex2_url = 'http://cseweb.ucsd.edu/classes/fa19/cse258-a/code/week3.py'

In [6]:
def gSave(url, path, filename):
    r = requests.get(url) #request url
    open(os.path.join(path, filename), 'wb').write(r.content) #save file in path

def parseData(fname, dataset = []):
    print('Reading data from', fname,'...')
    
    if re.search(r'\.gz$', fname):
        for line in gzip.open(fname, 'rt', errors = 'ignore'):
            yield ast.literal_eval(line)
        print('Done!')

    else:
        for l in open(fname):
            yield ast.literal_eval(l)
        print('Done')
        
def getData(src, dst):
    try:
            gSave(url = src, 
              path = dst,
              filename = src.split('/')[-1])
    except FileNotFoundError as e:
        os.mkdir(dst)
        gSave(url = src, 
              path = dst,
              filename = src.split('/')[-1])

In [7]:
fp = 'data'
getData(beer_url, fp)
getData(fb_url, fp)

fp = 'examples'
getData(ex1_url, fp)
getData(ex2_url, fp)

## Tasks — Diagnostics (week 2):

We’ll start by building a classifier that predicts whether a beer is highly alcoholic (ABV greater than 7 percent).
First, randomly shuffle the data and split it into 50%/50% train/test fractions.

In [8]:
fp = os.path.join('data', 'beer_50000.json')
beer = list(parseData(fp))        

Reading data from data/beer_50000.json ...
Done


In [9]:
# view the first element to get an idea of the data features
beer[0], len(beer)

({'review/appearance': 2.5,
  'beer/style': 'Hefeweizen',
  'review/palate': 1.5,
  'review/taste': 1.5,
  'beer/name': 'Sausa Weizen',
  'review/timeUnix': 1234817823,
  'beer/ABV': 5.0,
  'beer/beerId': '47986',
  'beer/brewerId': '10325',
  'review/timeStruct': {'isdst': 0,
   'mday': 16,
   'hour': 20,
   'min': 57,
   'sec': 3,
   'mon': 2,
   'year': 2009,
   'yday': 47,
   'wday': 0},
  'review/overall': 1.5,
  'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
  'user/profileName': 'stcules',
  'review/aroma': 2.0},
 50000)

1) We’ll use the *style* of the beer to predict its ABV. Construct a one-hot encoding of the beer style, for
those categories that appear in more than 1,000 reviews. You can build a mapping of categories to feature
indices as follows:

<code id = block>
    categoryCounts = defaultdict(int)
    for d in data:
        categoryCounts[d['beer/style']] += 1<br />
    categories = [c for c in categoryCounts if categoryCounts[c] > 1000]
    catID = dict(zip(list(categories),range(len(categories))))
</code>
<br />

Train a logistic regressor using this one-hot encoding to predict whether beers have an ABV greater than
7 percent (i.e., `d['beer/ABV'] > 7`). Train the classifier on the training set and report its performance
in terms of the accuracy and Balanced Error Rate (BER) on the test set, using a regularization constant
of $C = 10$. For all experiments use the `class_weight='balanced'` option (2 marks)

In [10]:
beer_styles = [b['beer/style'] for b in beer if 'beer/style' in b]
categoryCounts = dict(zip(beer_styles, np.zeros(len(beer_styles))))

for b in beer:
    categoryCounts[b['beer/style']] += 1
        

categories = [c for c in categoryCounts if categoryCounts[c] > 1000]
catID = np.array(list(zip(list(categories),range(len(categories)))))
catID_dict = dict(catID)
catID_dict

{'American Double / Imperial IPA': '0',
 'Rauchbier': '1',
 'American Pale Ale (APA)': '2',
 'American Porter': '3',
 'Russian Imperial Stout': '4',
 'American IPA': '5',
 'Fruit / Vegetable Beer': '6',
 'American Double / Imperial Stout': '7',
 'Rye Beer': '8',
 'Scotch Ale / Wee Heavy': '9',
 'English Pale Ale': '10',
 'Czech Pilsener': '11',
 'Old Ale': '12'}

In [12]:
X = [b['beer/style'] for b in beer if b['beer/style'] in catID]
y = [b['beer/ABV'] > 7 for b in beer if b['beer/style'] in catID]

ohe = OneHotEncoder(sparse = False)
X = ohe.fit_transform(np.array(X).reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 1/3)
logre = linear_model.LogisticRegression(C = 10, 
                                        class_weight = 'balanced')
preds = logre.fit(x_train, y_train).predict(x_test)

In [24]:
preds == True

array([ True, False,  True, ...,  True, False,  True])

In [41]:
preds == False

array([False,  True, False, ..., False,  True, False])

In [151]:
def tpr(y_pred, y_actual, to_print = True):
    tprate = sum((y_pred == True) == y_actual)/sum(y_pred)
    if to_print:
        print('Calculated a TPR of %f%%' % (tprate))
    return tprate

def fpr(y_pred, y_actual, to_print = True):
    fprate = sum((y_pred == False) == y_actual)/sum(y_pred == False)
    if to_print:
        print('Calculated a FPR of %f%%' % (fprate))
    return fprate
    
def ber(y_pred, y_actual):
    berate = 1 - (0.5 * (tpr(y_pred, y_actual, False) + fpr(y_pred, y_actual, False)))
    display_msg=('Using logistic regression on the One Hot Encoded beer styles to predict a beer ' +
                 'ABV avove 7.0%%, I predicted the\nBER to be  %f' % berate)
    print(display_msg)
    return berate

In [152]:
accuracy = logre.score(x_test, y_test)
display_msg = ('Using logistic regression on the One Hot Encoded beer styles to predict a beer ABV avove 7.0%, '+
               f'I predicted the \naccuracy to be  %f' % accuracy)
print(display_msg)

Using logistic regression on the One Hot Encoded beer styles to predict a beer ABV avove 7.0%, I predicted the 
accuracy to be  0.930928


In [153]:
display(pd.DataFrame({'y_test':y_test,
                              'prediction':preds}))
display(pd.DataFrame({'tpr':[tpr(preds, y_test)],
                      'fpr':[fpr(preds, y_test)],
                      'ber':[ber(preds, y_test)]}))

,y_test,prediction
0,True,True
1,False,False
2,True,True
3,True,True
4,False,False
...,...,...
10969,True,True
10970,False,False
10971,True,True
10972,True,False


Calculated a TPR of 1.861177%
Calculated a FPR of 0.138195%
Using logistic regression on the One Hot Encoded beer styles to predict a beer ABV avove 7.0%, I predicted the
BER to be  0.000314


,tpr,fpr,ber
0,1.861177,0.138195,0.000314


2) Extend your model to include two additional features: <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1) a vector of five ratings (`review/aroma`,
`review/overall`, etc.); and <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2) the review length (in characters).<br> The length feature should be scaled to
be between 0 and 1 by dividing by the maximum length. Using the same value of C from the previous
question, report the BER of the new classifier (1 mark).

3) Implement a complete regularization pipeline with the balanced classifier. Split your test data from above
in half so that you have 50%/25%/25% train/validation/test fractions. Consider values of $C$ in the range
$\{10−6, 10−5, 10−4, 10−3\}$. Report (or plot) the train, validation, and test BER for each value of $C$. Based
on these values, which classifier would you select (in terms of generalization performance) and why (1
mark)?

4) **(CSE158 only)** An *ablation study* measures the marginal benefit of various features by re-training the
model with one feature ‘ablated’ (i.e., deleted) at a time. Considering each of the three features in your
classifier above (i.e., beer style, ratings, and length), report the BER with only the other two features
and the third deleted (1 mark).


## Tasks (Community Detection):
Download the Facebook ego-network data.


6) How many connected components are in the graph, and how many nodes are in the largest connected
component (1 mark)?

Next we’ll implement a ‘greedy’ version of normalized cuts, using <b><i>just the largest connected component</i></b>
found above. First, split it into two equal halves, just by taking the 50% of nodes with the lowest and 50%
with the highest IDs.

7) What is the normalized-cut cost of the 50/50 split you found above (1 mark)?

Now we’ll implement our greedy algorithm as follows: during each step, we’ll move one node from one
cluster to the other, choosing whichever move *minimizes the resulting normalized cut cost* (in case of a tie, pick
the node with the lower ID). Repeat this until the cost can’t be reduced any further.

8) What are the elements of the split, and what is its normalized cut cost (1 mark)?